# Welcome to the Lab 🥼🧪
## How do I download data as a CSV?

In this notebook, we will be building some basic intuition around how search works, broken into basic and advanced topics. 

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/introduction/download_data.ipynb)

In [1]:
import os
import sys
import json
import subprocess
from datetime import datetime
from urllib.request import urlopen

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [2]:
import parcllabs
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

Parcl Labs Version: 0.2.0


In [3]:
# Initialize the Parcl Labs client
client = ParclLabsClient(api_key, limit=12) # lets set our own limit for all calls

In [7]:
# lets get some sample markets to download data for, everything starts with search
# lets get the top 12 most populous metros in the country
markets = client.search_markets.retrieve(
    location_type='CBSA',
    sort_by='TOTAL_POPULATION',
    sort_order='DESC',
    as_dataframe=True
)

# lets store the parcl_ids for use
parcl_ids = markets['parcl_id'].tolist()

markets.head(5)


,parcl_id,country,geoid,state_fips_code,name,state_abbreviation,region,location_type,total_population,median_income,parcl_exchange_market,pricefeed_market,case_shiller_10_market,case_shiller_20_market
0,2900187,USA,35620,None,"New York-Newark-Jersey City, Ny-Nj-Pa",None,None,CBSA,19908595,93610,0,1,1,1
1,2900078,USA,31080,None,"Los Angeles-Long Beach-Anaheim, Ca",None,None,CBSA,13111917,89105,0,1,1,1
2,2899845,USA,16980,None,"Chicago-Naperville-Elgin, Il-In-Wi",None,None,CBSA,9566955,85087,0,1,1,1
3,2899734,USA,19100,None,"Dallas-Fort Worth-Arlington, Tx",None,None,CBSA,7673379,83398,0,1,0,1
4,2899967,USA,26420,None,"Houston-The Woodlands-Sugar Land, Tx",None,None,CBSA,7142603,78061,0,1,0,0


In [10]:
# we want the last year of all data for the purposes of this notebook
START_DATE = '2023-01-01'

In [12]:
# download supply/demand metrics for the top 12 metros
supply_demand = client.market_metrics_housing_event_counts.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

supply_demand.head(5)

|████████████████████████████████████████| 12/12 [100%] in 1.8s (6.79/s) 


,date,sales,new_listings_for_sale,new_rental_listings,parcl_id
0,2024-04-01,17622,14346,21345,2900187
1,2024-03-01,19888,13202,21683,2900187
2,2024-02-01,18579,12013,20542,2900187
3,2024-01-01,20792,11353,22428,2900187
4,2023-12-01,19747,6448,18110,2900187


In [ ]:
# save to csv locally
supply_demand.to_csv('supply_demand.csv', index=False)

In [13]:
# download market prices for the top 12 metros
prices = client.market_metrics_housing_event_prices.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

prices.head()

|████████████████████████████████████████| 12/12 [100%] in 1.7s (7.03/s) 


,date,price_median_sales,price_median_new_listings_for_sale,price_median_new_rental_listings,price_standard_deviation_sales,price_standard_deviation_new_listings_for_sale,price_standard_deviation_new_rental_listings,price_percentile_20th_sales,price_percentile_20th_new_listings_for_sale,price_percentile_20th_new_rental_listings,...,price_per_square_foot_standard_deviation_sales,price_per_square_foot_standard_deviation_new_listings_for_sale,price_per_square_foot_standard_deviation_new_rental_listings,price_per_square_foot_percentile_20th_sales,price_per_square_foot_percentile_20th_new_listings_for_sale,price_per_square_foot_percentile_20th_new_rental_listings,price_per_square_foot_percentile_80th_sales,price_per_square_foot_percentile_80th_new_listings_for_sale,price_per_square_foot_percentile_80th_new_rental_listings,parcl_id
0,2024-04-01,635000,750000,3485,165776,267922,823,490000,580000,2762,...,157.25,185.36,2.21,282.04,297.88,2.73,507.35,587.15,6.71,2900187
1,2024-03-01,610000,749900,3500,152541,269117,799,477000,559900,2800,...,151.42,190.59,2.17,269.94,293.14,2.80,487.69,589.81,6.76,2900187
2,2024-02-01,600000,749995,3400,155420,284337,781,465000,550000,2720,...,152.43,198.41,2.34,267.24,295.18,2.68,485.89,613.21,6.32,2900187
3,2024-01-01,610000,725000,3493,157576,248951,785,476900,549000,2800,...,150.31,196.54,2.16,270.90,293.77,2.70,485.15,600.00,6.45,2900187
4,2023-12-01,600000,650000,3451,153970,206646,749,466000,499000,2800,...,144.10,184.22,2.27,268.47,273.44,2.69,474.33,550.00,6.52,2900187


In [ ]:
# save to csv locally
prices.to_csv('prices.csv', index=False)

In [14]:
# download total housing stock for the top 12 metros
# i.e. how many single family homes, condos, townhomes are there in each market
housing_stock = client.market_metrics_housing_stock.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True
)

housing_stock.head()


|████████████████████████████████████████| 12/12 [100%] in 1.6s (7.40/s) 


,date,single_family,condo,townhouse,other,all_properties,parcl_id
0,2024-04-01,2802979,968157,77448,1585155,5433739,2900187
1,2024-03-01,2802961,967776,77411,1584989,5433137,2900187
2,2024-02-01,2802941,967347,77359,1584803,5432450,2900187
3,2024-01-01,2802920,966811,77328,1584625,5431684,2900187
4,2023-12-01,2802882,966335,77301,1584461,5430979,2900187


In [ ]:
# save to csv locally
housing_stock.to_csv('housing_stock.csv', index=False)

In [15]:
# download all cash purchasing behavior for top 12 metros
cash_purchases = client.market_metrics_all_cash.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

cash_purchases.head()

|████████████████████████████████████████| 12/12 [100%] in 1.6s (7.33/s) 


,date,count,pct_all_cash,parcl_id
0,2024-04-01,4832,27.42,2900187
1,2024-03-01,6070,30.52,2900187
2,2024-02-01,5594,30.11,2900187
3,2024-01-01,6473,31.13,2900187
4,2023-12-01,5742,29.08,2900187


In [ ]:
# save to csv locally
cash_purchases.to_csv('cash_purchases.csv', index=False)

In [16]:
# Download gross yields for the top 12 metros
gross_yields = client.rental_market_metrics_gross_yield.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

gross_yields.head()

|████████████████████████████████████████| 12/12 [100%] in 1.7s (7.04/s) 


,date,pct_gross_yield,parcl_id
0,2024-04-01,5.58,2900187
1,2024-03-01,5.60,2900187
2,2024-02-01,5.44,2900187
3,2024-01-01,5.78,2900187
4,2023-12-01,6.37,2900187


In [ ]:
# save to csv locally
gross_yields.to_csv('gross_yields.csv', index=False)

In [17]:
# download new listings for rent for the top 12 metros
new_rental_listings = client.rental_market_metrics_new_listings_for_rent_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

new_rental_listings.head()

|████████████████████████████████████████| 12/12 [100%] in 1.6s (7.53/s) 


,date,rolling_7_day,rolling_30_day,rolling_60_day,rolling_90_day,parcl_id
0,2024-05-13,5715,24088,50538,75867,2900187
1,2024-05-06,6510,24384,50736,76104,2900187
2,2024-04-29,4761,24226,50236,75280,2900187
3,2024-04-22,5540,24834,50476,75612,2900187
4,2024-04-15,5889,25205,50660,76429,2900187


In [ ]:
# save to csv locally
new_rental_listings.to_csv('new_rental_listings.csv', index=False)

In [18]:
# Download rental unit concentration for the top 12 metros
rental_unit_concentration = client.rental_market_metrics_rental_units_concentration.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

rental_unit_concentration.head()

|████████████████████████████████████████| 12/12 [100%] in 1.7s (7.24/s) 


,date,rental_units,total_units,pct_rental_concentration,parcl_id
0,2024-04-01,719023,5433706,13.23,2900187
1,2024-03-01,718929,5433104,13.23,2900187
2,2024-02-01,718804,5432417,13.23,2900187
3,2024-01-01,718634,5431651,13.23,2900187
4,2023-12-01,718491,5430946,13.23,2900187


In [ ]:
# save to csv locally
rental_unit_concentration.to_csv('rental_unit_concentration.csv', index=False)

In [19]:
# Download new listings for sale for the top 12 metros
new_sale_listings = client.for_sale_market_metrics_new_listings_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

new_sale_listings.head()

|████████████████████████████████████████| 12/12 [100%] in 1.6s (7.48/s) 


,date,rolling_7_day,rolling_30_day,rolling_60_day,rolling_90_day,parcl_id
0,2024-05-13,3457,15922,33141,49006,2900187
1,2024-05-06,4001,16477,33348,49021,2900187
2,2024-04-29,3616,16589,33245,48209,2900187
3,2024-04-22,3824,15979,32814,47223,2900187
4,2024-04-15,3942,15726,31928,46442,2900187


In [ ]:
# save to csv locally
new_sale_listings.to_csv('new_sale_listings.csv', index=False)

In [20]:
# download investor supply/demand metrics for the top 12 metros
investor_supply_demand = client.investor_metrics_housing_event_counts.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

investor_supply_demand.head()

|████████████████████████████████████████| 12/12 [100%] in 1.5s (7.90/s) 


,date,acquisitions,dispositions,new_listings_for_sale,new_rental_listings,parcl_id
0,2024-04-01,2455,2036,1334,774,2900187
1,2024-03-01,3340,2191,1297,818,2900187
2,2024-02-01,3222,2073,1223,725,2900187
3,2024-01-01,3743,2331,1194,942,2900187
4,2023-12-01,3236,2094,774,622,2900187


In [ ]:
# save to csv locally
investor_supply_demand.to_csv('investor_supply_demand.csv', index=False)

In [21]:
# Download investor pricing activity
investor_pricing = client.investor_metrics_housing_event_prices.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

investor_pricing.head()

|████████████████████████████████████████| 12/12 [100%] in 1.6s (7.60/s) 


,date,price_median_acquisitions,price_median_dispositions,price_median_new_listings_for_sale,price_median_new_rental_listings,price_per_square_foot_median_acquisitions,price_per_square_foot_median_dispositions,price_per_square_foot_median_new_listings_for_sale,price_per_square_foot_median_new_rental_listings,parcl_id
0,2024-04-01,550000,720000,949000,3850,327.14,398.50,444.52,3.36,2900187
1,2024-03-01,560000,682375,888000,3600,320.47,381.47,421.74,3.06,2900187
2,2024-02-01,552800,672000,889500,3600,315.64,375.36,429.61,2.91,2900187
3,2024-01-01,596500,661250,849450,3825,327.20,388.96,400.35,3.13,2900187
4,2023-12-01,555000,665000,750000,3300,318.51,367.07,387.42,2.74,2900187


In [ ]:
# save to csv locally
investor_pricing.to_csv('investor_pricing.csv', index=False)

In [22]:
# Download total investor ownership of all homes for the top 12 metros
total_investor_ownership = client.investor_metrics_housing_stock_ownership.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True
)

total_investor_ownership.head()

|████████████████████████████████████████| 12/12 [100%] in 1.6s (7.69/s) 


,date,count,pct_ownership,parcl_id
0,2024-04-01,351103,6.46,2900187
1,2024-03-01,350073,6.44,2900187
2,2024-02-01,348873,6.42,2900187
3,2024-01-01,347800,6.40,2900187
4,2023-12-01,346469,6.38,2900187


In [ ]:
# save to csv locally
total_investor_ownership.to_csv('total_investor_ownership.csv', index=False)

In [23]:
# download investor new listings activity
investor_new_listings = client.investor_metrics_new_listings_for_sale_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

investor_new_listings.head()

|████████████████████████████████████████| 12/12 [100%] in 1.6s (7.72/s) 


,date,period,counts,pct_for_sale_market,parcl_id
0,2024-05-13,rolling_7_day,353,10.21,2900187
1,2024-05-13,rolling_30_day,1530,9.61,2900187
2,2024-05-13,rolling_60_day,3174,9.58,2900187
3,2024-05-13,rolling_90_day,4811,9.82,2900187
4,2024-05-06,rolling_7_day,375,9.37,2900187


In [ ]:
# save to csv locally
investor_new_listings.to_csv('investor_new_listings.csv', index=False)

In [24]:
# download investor purchase to sale ratio, i.e. net buyers or sellers for the top 12 metros
purchase_to_sale_ratio = client.investor_metrics_purchase_to_sale_ratio.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,
    start_date=START_DATE
)

purchase_to_sale_ratio.head()

|████████████████████████████████████████| 12/12 [100%] in 1.7s (7.18/s) 


,date,purchase_to_sale_ratio,parcl_id
0,2024-04-01,1.21,2900187
1,2024-03-01,1.52,2900187
2,2024-02-01,1.55,2900187
3,2024-01-01,1.61,2900187
4,2023-12-01,1.55,2900187


In [ ]:
# save to csv locally
purchase_to_sale_ratio.to_csv('purchase_to_sale_ratio.csv', index=False)

In [27]:
# download institutional supply/demand metrics for the top 12 metros
institutional_supply_demand = client.portfolio_metrics_sf_housing_event_counts.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True,    
    portfolio_size='PORTFOLIO_1000_PLUS'
)

institutional_supply_demand.head()

|████████████████████████████████████████| 12/12 [100%] in 1.6s (7.64/s) 


,date,acquisitions,dispositions,new_listings_for_sale,new_rental_listings,parcl_id,portfolio_size
0,2024-04-01,12,9,11,0,2900187,PORTFOLIO_1000_PLUS
1,2024-03-01,21,7,31,4,2900187,PORTFOLIO_1000_PLUS
2,2024-04-01,14,19,20,75,2900078,PORTFOLIO_1000_PLUS
3,2024-03-01,6,14,27,77,2900078,PORTFOLIO_1000_PLUS
4,2024-04-01,78,50,54,105,2899845,PORTFOLIO_1000_PLUS


In [ ]:
# save to csv locally
institutional_supply_demand.to_csv('institutional_supply_demand.csv', index=False)

In [30]:
# Get institutional total ownership for the top 12 metros
institional_ownership = client.portfolio_metrics_sf_housing_stock_ownership.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True
)

institional_ownership.head()

|████████████████████████████████████████| 12/12 [100%] in 1.5s (7.84/s) 


,date,count_portfolio_2_to_9,count_portfolio_10_to_99,count_portfolio_100_to_999,count_portfolio_1000_plus,count_all_portfolios,pct_sf_housing_stock_portfolio_2_to_9,pct_sf_housing_stock_portfolio_10_to_99,pct_sf_housing_stock_portfolio_100_to_999,pct_sf_housing_stock_portfolio_1000_plus,pct_sf_housing_stock_all_portfolios,parcl_id
0,2024-04-01,141252,7891,1078,2152,152373,5.04,0.28,0.04,0.08,5.44,2900187
1,2024-03-01,135863,7295,1033,1998,146189,4.85,0.26,0.04,0.07,5.22,2900187
2,2024-04-01,195506,10711,868,4348,211433,9.78,0.54,0.04,0.22,10.58,2900078
3,2024-03-01,194888,10628,861,4339,210716,9.76,0.53,0.04,0.22,10.55,2900078
4,2024-04-01,96999,16044,2848,10672,126563,4.81,0.79,0.14,0.53,6.27,2899845


In [ ]:
# save to csv locally
institional_ownership.to_csv('institional_ownership.csv', index=False)

In [32]:
# Get institutional rental listings for the top 12 metros
institutional_rental_listings = client.portfolio_metrics_sf_new_listings_for_rent_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True
)

institutional_rental_listings.head()

|████████████████████████████████████████| 12/12 [100%] in 1.4s (8.29/s) 


,date,period,counts,pct_sf_for_rent_market,parcl_id,portfolio_size
0,2024-05-13,rolling_7_day,78,13.81,2900187,ALL_PORTFOLIOS
1,2024-05-13,rolling_30_day,348,15.80,2900187,ALL_PORTFOLIOS
2,2024-05-13,rolling_60_day,702,15.99,2900187,ALL_PORTFOLIOS
3,2024-05-13,rolling_90_day,1091,16.00,2900187,ALL_PORTFOLIOS
4,2024-05-06,rolling_7_day,101,17.47,2900187,ALL_PORTFOLIOS


In [ ]:
# save to csv locally
institutional_rental_listings.to_csv('institutional_rental_listings.csv', index=False)

In [34]:
# Get institutional new listings for sale for the top 12 metros
institutional_sale_listings = client.portfolio_metrics_new_listings_for_sale_rolling_counts.retrieve_many(
    parcl_ids=parcl_ids,
    as_dataframe=True
)

institutional_sale_listings.head()

|████████████████████████████████████████| 12/12 [100%] in 1.5s (7.87/s) 


,date,period,counts,pct_sf_for_sale_market,parcl_id,portfolio_size
0,2024-05-13,rolling_7_day,187,11.35,2900187,ALL_PORTFOLIOS
1,2024-05-13,rolling_30_day,772,10.38,2900187,ALL_PORTFOLIOS
2,2024-05-13,rolling_60_day,1599,10.77,2900187,ALL_PORTFOLIOS
3,2024-05-13,rolling_90_day,2365,11.02,2900187,ALL_PORTFOLIOS
4,2024-05-06,rolling_7_day,162,8.92,2900187,ALL_PORTFOLIOS


In [ ]:
# save to csv locally
institutional_sale_listings.to_csv('institutional_sale_listings.csv', index=False)